In [15]:
import socket
import hashlib
import os
import time
import threading
import sys
import Crypto.Cipher.AES as AES
from Crypto.PublicKey import RSA

def create_sess_key_and_encrypt(getpbk):
    
    server_public_key = RSA.importKey(getpbk) # client로 부터 받은 public key를
                                              # 문자열로 인코딩 -> RSA 키값으로 가져옴
    # 세션키 생성
    sess_key = os.urandom(16)
    aesEncrypt = AES.new(sess_key,AES.MODE_CTR,counter = lambda:sess_key)
    sess_key_aesEncrypt = aesEncrypt.encrypt(sess_key)
    sess_hash_object = hashlib.sha1(sess_key_aesEncrypt)
    sess_hex_digest = sess_hash_object.hexdigest()
    
    E = server_public_key.encrypt(sess_key_aesEncrypt,16)
    return sess_hex_digest,E

#server address and port number input from admin
host= "0.0.0.0"
port = 9999
#boolean for checking server and port
check = False

try:
    #setting up socket
    server = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
    server.bind((host,port))
    server.listen(5)
    check = True
except BaseException:
    print "-----Check Server Address or Port-----"
    check = False

if check==True:
    shutdown = False
    print("\n-----Server Started. Waiting for client-----\n")
    client,address = server.accept()
    print("Client is connected. Client's address :",address)
    print("\n-----Waiting for public key hash-----\n")
    
    getpbk = client.recv(2048)
    
    hash_object = hashlib.sha1(getpbk)
    hex_digest = hash_object.hexdigest()
    
    if getpbk != "":
        print("\n-----Public Key received from client-----\n")
        print(getpbk)
        client.send("YES") # Public_key 정상적으로 받았다고, client에게 'YES' 전송
        
        gethash = client.recv(1024)
        print("\n-----Public Key Hash received from client-----\n"+gethash)
    
    #client로 부터 받은 pbk를 해쉬한 값과 client로 부터 받은 pbk 해쉬값이 같으면
    if hex_digest == gethash: 
        
        sess_hex_digest, encryp_pbk_and_sess_k = create_sess_key_and_encrypt(getpbk)
        print("\n-----Session Key Hash-----\n"+sess_hex_digest) 
        print( "\n-----Public key encrypted with session key-----\n"+str(encryp_pbk_and_sess_k))
        client.send(str(encryp_pbk_and_sess_k)) # client에서 msg로 받음
        
        print("\n-----Handshake complete-----\n")
        
        newmess = client.recv(1024)
        print("\n-----Encrypted Message from Client-----\n"+newmess)
        
        decoded = newmess.decode("hex")
        key = sess_hex_digest[:16]
        aesDecrypt = AES.new(key, AES.MODE_CTR,counter=lambda : key)
        dMsg = aesDecrypt.decrypt(decoded)
        print("\nDecrypt Message received from client** " + time.ctime() + " : " + dMsg +"\n")
        
        mess = raw_input("\nMessage to client : ")
        if mess != "":
            aesEncrypt = AES.new(key,AES.MODE_CTR,counter=lambda : key)
            eMsg = aesEncrypt.encrypt(mess).encode("hex").upper()
            if eMsg != "":
                print("Encrypted Message to client : "+eMsg)
            client.send(eMsg)
        client.close()
    else:
        print("\n-----Public key hash does not match-----\n")


-----Server Started. Waiting for client-----

("Client is connected. Client's address :", ('127.0.0.1', 61252))

-----Waiting for public key hash-----


-----Public Key received from client-----

-----BEGIN PUBLIC KEY-----
MIGfMA0GCSqGSIb3DQEBAQUAA4GNADCBiQKBgQDKcuaJFmDY/2WHBui0V4AyjVdu
F6Zy8q/ZvNfiDsXTiFqYmCYfWlQXQAZwgX18tkgPgnKlFid/74nfh1c8jMd0YR9U
egh9I/aIu6nb/++hUyXwH68h1V7fu+4snmcInL0HN0Tfv35aUUEW4zZcPbnW73LT
EXvyle0lzSB3u77HDwIDAQAB
-----END PUBLIC KEY-----

-----Public Key Hash received from client-----
6e5d10ae14ce6cfc8cb140a9b6f5febce8834225

-----Session Key Hash-----
345a761dca3193350b2b90a7b31e4b981972ff23

-----Public key encrypted with session key-----
('{K\xba\x85\xb8\xc3\xf9\x8c\x15\x1eZ\x0e\xab\xbe\x1e+\x18\xe3;\x97\n\xfeO\xc8\xa2aO"^\xff\xba\xcf\xbb\xf5]r5\xed\xa3\x8a\xd4w\xdf\x07\xec\x01JV\x9c\x1a\xd2\xb4\x13\x98{\x8c\xbc\xf0o\xf3\x7f\x04\x95\xc0s\x9a3\xd6\x05lB\x8c\xe1\xa7\xea\xb5\xa2\xe8\x18ey6\x87\xac8\xeet\xc4)@\xa2\xd7[\x06bXU\xda\xbd\x90\xf4\xe0V\xfb\xbf\xa24\